In [9]:
import os
import asyncio
from dotenv import load_dotenv
from langchain.agents import initialize_agent, Tool, AgentType
from langchain.chat_models import ChatOpenAI
from mcp.client.stdio import stdio_client
from mcp.schema import CallToolRequest

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY not found in environment variables.")

# -------------------------------
# MCP Client Utilities
# -------------------------------
async def mcp_call(tool_name: str, arguments: dict):
    """Call a tool on the Airbnb MCP server."""
    async with stdio_client(["npx", "-y", "@openbnb/mcp-server-airbnb"]) as (read, write):
        # Send the CallTool request
        await write(
            CallToolRequest(name=tool_name, arguments=arguments).model_dump(mode="json")
        )

        # Read and return server response
        async for message in read:
            if message.get("result"):
                return message["result"]

# -------------------------------
# LangChain Tool Wrappers
# -------------------------------
def make_airbnb_search_tool():
    async def _search(location: str, checkin: str, checkout: str, adults: int, max_price: int):
        args = {
            "location": location,
            "checkin": checkin,
            "checkout": checkout,
            "adults": adults,
            "max_price": max_price
        }
        return await mcp_call("airbnb_search", args)
    return Tool(
        name="airbnb_search",
        func=lambda **kwargs: asyncio.run(_search(**kwargs)),
        description="Search Airbnb listings given location, dates, number of adults, and max price."
    )

def make_airbnb_listing_details_tool():
    async def _details(listing_id: str):
        args = {"id": listing_id}
        return await mcp_call("airbnb_listing_details", args)
    return Tool(
        name="airbnb_listing_details",
        func=lambda **kwargs: asyncio.run(_details(**kwargs)),
        description="Get detailed information about an Airbnb listing by ID."
    )

# -------------------------------
# Main Agent
# -------------------------------
def run_airbnb_agent(prompt: str):
    llm = ChatOpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)

    tools = [
        make_airbnb_search_tool(),
        make_airbnb_listing_details_tool()
    ]

    agent = initialize_agent(
        tools,
        llm,
        agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
        verbose=True
    )

    response = agent.run(prompt)
    print("Agent response:", response)

# -------------------------------
# Example Run
# -------------------------------
if __name__ == "__main__":
    prompt = (
        "Search Airbnb listings in Bengaluru for 2 adults, "
        "check-in 2025-09-01, check-out 2025-09-05, "
        "under ₹5,000 per night. Then pick one listing "
        "and get its details."
    )
    run_airbnb_agent(prompt)


ModuleNotFoundError: No module named 'mcp.schema'